# Image Cluster Notebook

In [1]:
from ipyleaflet import basemaps
from ipyleaflet import TileLayer
import ipysheet
from IPython.display import Markdown
import ipywidgets
from joblib import cpu_count
import leafmap
import numpy
import pandas
from pathlib import Path
import sys
from osgeo import gdal
from sklearn import cluster

In [2]:
inFile = Path('/explore/nobackup/people/rlgill/innovation-lab-repositories/ImageCluster/my4326.tif')
redBandId = 3
greenBandId = 2
blueBandId = 1

# inFile = Path('/explore/nobackup/people/mcarrol2/LCLUC_Senegal/ForKonrad/Tappan26_WV03_20210314_M1BS_10400100676F0900_isos.tif')
# redBandId = 1
# greenBandId = 1
# blueBandId = 1

# inFile = Path('/explore/nobackup/people/mcarrol2/LCLUC_Senegal/ForKonrad/Tappan26_WV03_20210314_M1BS_10400100676F0900_data.tif')
# redBandId = 5
# greenBandId = 4
# blueBandId = 2

noDataValue = -9999.0 or None
outDirectory = inFile.parent


In [3]:
# ----------------------------------------------------------------------------
# handleClick
# ----------------------------------------------------------------------------
def handleClick(change: dict) -> None:
    
    with output:
        
        if change.new == 'Next':

            nn = updateList(list(sl.options), list(sl.value))
            updateDict('N')
            sl.options = nn
            bt.value='Select:'

        if change.new == 'Done':
            updateDict('D')

        if change.new == 'Start Over':
        
            sl.options = opts
            updateDict('S')
            bt.value = 'Select:'

# ----------------------------------------------------------------------------
# relabel
# ----------------------------------------------------------------------------
def relabel(labelArray: numpy.ndarray, lookup: dict) -> numpy.ndarray:
    
    newLab = labelArray.copy()
    
    for k, v in lookup.items():

        if len(v)==1 & k==v[0]:
            pass
        else:
            newLab = numpy.where(numpy.isin(newLab, v), k, newLab)
    
    return newLab
            
# ----------------------------------------------------------------------------
# updateDict
# ----------------------------------------------------------------------------
def updateDict(op: str) -> None:
    
    if op == 'N':
        
        key = list(sl.value)[0]
        table[key] = list(sl.value)
        # print('Re-grouping : ', table)
    
    if op == 'D':
    
        if len(sl.options) > 0:
            
            key = list(sl.options)[0]
            table[key] = list(sl.options)
        
        print('Final Groups : ', table)
    
    if op == 'S':
        
        table.clear()
    
# ----------------------------------------------------------------------------
# updateList
# ----------------------------------------------------------------------------
def updateList(old: list, out: list) -> list:
    return [ele for ele in old if ele not in out]


In [4]:
# # ----------------------------------------------------------------------------
# # Class ImageHelper
# #
# # TODO: add accessors using @property.
# # ----------------------------------------------------------------------------
# class ImageHelper(object):
    
#     # ------------------------------------------------------------------------
#     # init
#     # ------------------------------------------------------------------------
#     def __init__(self):
        
#         self._dataset: gdal.Dataset = None
#         self._inputFile: Path = None
#         self._redBandId: int = 1
#         self._greenBandId: int = 1
#         self._blueBandId: int = 1
#         self._redBand: numpy.ndarray = None
#         self._greenBand: numpy.ndarray = None
#         self._blueBand: numpy.ndarray = None
#         self._noDataValue: float = -9999.0
#         self._minValue: float = sys.float_info.max
#         self._maxValue: float = sys.float_info.min
  
#     # ------------------------------------------------------------------------
#     # initFromDataset
#     # ------------------------------------------------------------------------
#     def initFromDataset(self, 
#                         dataset: gdal.Dataset,
#                         noDataValue: float,
#                         redBandId: int = 1,
#                         greenBandId: int = 1,
#                         blueBandId: int = 1,
#                        ) -> None:
        
#         self._dataset = dataset
        
#         self._completeInitialization(noDataValue, 
#                                      redBandId, 
#                                      greenBandId, 
#                                      blueBandId)
        
#     # ------------------------------------------------------------------------
#     # initFromFile
#     # ------------------------------------------------------------------------
#     def initFromFile(self, 
#                      inputFile: Path,
#                      noDataValue: float,
#                      redBandId: int = 1,
#                      blueBandId: int = 1,
#                      greenBandId: int = 1,
#                     ) -> None:
        
#         # TODO: check validity and comment.
#         self._inputFile = inputFile
#         self._dataset: gdal.Dataset = gdal.Open(str(self._inputFile))
        
#         self._completeInitialization(noDataValue, 
#                                      redBandId, 
#                                      greenBandId, 
#                                      blueBandId)
        
#     # ------------------------------------------------------------------------
#     # completeInitialization
#     # ------------------------------------------------------------------------
#     def _completeInitialization(self,
#                                 noDataValue: float,
#                                 redBandId: int = 1,
#                                 greenBandId: int = 1,
#                                 blueBandId: int = 1,
#                                ) -> None:
        
#         self._redBandId = redBandId
#         self._greenBandId = greenBandId
#         self._blueBandId = blueBandId
        
#         # Generate overviews, if they do not exist.
#         if self._dataset.GetRasterBand(1).GetOverviewCount() == 0:
#             dummy =self. _dataset.BuildOverviews()

#         # ---
#         # Read the bands.
#         # ---
#         self._redBand: numpy.ndarray = \
#             self._dataset.GetRasterBand(self._redBandId).ReadAsArray()
        
#         self._greenBand: numpy.ndarray = \
#             self._dataset.GetRasterBand(self._greenBandId).ReadAsArray()
        
#         self._blueBand: numpy.ndarray = \
#             self._dataset.GetRasterBand(self._blueBandId).ReadAsArray()
        
#         # ---
#         # Initialize the no-data value.
#         # ---
#         self._noDataValue = self._dataset.GetRasterBand(self._redBandId). \
#             GetNoDataValue() or noDataValue
       
#         # ---
#         # Compute the minimum and maximum pixels values to help the renderer.
#         # ---
#         forExtremes = self._redBand[self._redBand != self._noDataValue]
#         self._minValue: float = forExtremes.min()
#         self._maxValue: float = forExtremes.max()

#     # ------------------------------------------------------------------------
#     # getCorners
#     #
#     # Why is this not built into gdal?
#     # ------------------------------------------------------------------------
#     def getCorners(self):
        
#         minx, xres, xskew, maxy, yskew, yres = \
#             self._dataset.GetGeoTransform()
        
#         maxx = minx + (self._dataset.RasterXSize * xres)
#         miny = maxy + (self._dataset.RasterYSize * yres)

#         return (minx, miny, maxx, maxy)
    
#     # ------------------------------------------------------------------------
#     # getRgbIndexList
#     # ------------------------------------------------------------------------
#     def getRgbIndexList(self) -> list:
        
#         return [self._redBandId, self._greenBandId, self._blueBandId]
    
#     # ------------------------------------------------------------------------
#     # getRgbBands
#     # ------------------------------------------------------------------------
#     def getRgbBands(self) -> list:
        
#         return [self._redBand, self._greenBand, self._blueBand]

#     # ------------------------------------------------------------------------
#     # __str__
#     # ------------------------------------------------------------------------
#     def __str__(self):
        
#         return ('Input file: ' + str(self._inputFile) + 
#                 '\nMin. pixel: ' + str(self._minValue) +
#                 '\nMax. pixel: ' + str(self._maxValue) + 
#                 '\nNo-data value: ' + str(self._noDataValue) + 
#                 '\nRed band index: ' + str(self._redBandId) +
#                 '\nGreen band index: ' + str(self._greenBandId) +
#                 '\nBlue band index: ' + str(self._blueBandId) +
#                 '\nCorners: ' + str(self.getCorners())
#                )
    

In [5]:
# ----------------------------------------------------------------------------
# Class Clusterer
# ----------------------------------------------------------------------------
class Clusterer(object):

    # ------------------------------------------------------------------------
    # getClusters
    # ------------------------------------------------------------------------
    @staticmethod
    def getClusters(bands: list, numClusters: int=5) -> numpy.ndarray:

        img = numpy.moveaxis(bands, 0, -1)
        img1d = img.reshape(-1, img.shape[-1])

        cl = cluster.MiniBatchKMeans(n_clusters=numClusters,
                                     random_state=0,
                                     batch_size=256*cpu_count()
                                    )
        
        model = cl.fit(img1d)
        imgCl = model.labels_
        imgCl = imgCl.reshape(img[:,:,0].shape)

        return imgCl

    # ------------------------------------------------------------------------
    # labelsToGeotiff
    # These renderers seem to write tiles or pyramids to disk.  
    # To make rendering code easier, write the labels as a geotiff; then the
    # renderer will not need to do it.
    # ------------------------------------------------------------------------
    @staticmethod
    def labelsToGeotiff(referenceDs: gdal.Dataset,
                        labelsFile: Path, 
                        labels: numpy.ndarray) -> gdal.Dataset:

        if labelsFile.exists():
            labelsFile.unlink()
            
        labelsDs = gdal.GetDriverByName('GTiff').Create( \
                        str(labelsFile),
                        xsize=referenceDs.RasterXSize,
                        ysize=referenceDs.RasterYSize,
                        eType=gdal.GDT_Float32
                   )

        labelsDs.SetSpatialRef(referenceDs.GetSpatialRef())
        labelsDs.SetGeoTransform(referenceDs.GetGeoTransform())
        outBand = labelsDs.GetRasterBand(1)
        outBand.WriteArray(labels)
        outBand.ComputeStatistics(0)  # For min, max used in color table
        labelsDs.FlushCache()
        labelsDs.BuildOverviews()

        return labelsDs


In [6]:
# ---
# Ingest the input image.
# ---
inHelper = ImageHelper()

inHelper.initFromFile(inputFile=inFile, 
                      noDataValue=noDataValue, 
                      redBandId=redBandId, 
                      greenBandId=greenBandId, 
                      blueBandId=blueBandId)

# ---
# Create the clusters, put them into a Geotiff, then ingest the image.
# ---
labels = Clusterer.getClusters(bands=inHelper.getRgbBands(), numClusters=30)
labelsFile = outDirectory / (inFile.stem + '-labels' + inFile.suffix)
labelsDs = Clusterer.labelsToGeotiff(inHelper._dataset, labelsFile, labels)
lHelper = ImageHelper()
lHelper.initFromDataset(labelsDs, noDataValue)

In [19]:
corners = inHelper.getCorners()

# ---
# The only reason used leafmap and not ipyleaflet directly is to get the
# inspector tool under the wrench button on leafmap's map.
# ---
m = leafmap.Map(fullscreen_control=False,
                layers_control=True,
                search_control=False,
                draw_control=False,
                measure_control=False,
                scale_control=False,
                toolbar_control=True,
                center=[corners[1], corners[0]])

m.fit_bounds([[corners[1], corners[0]], [corners[3], corners[2]]])

m.add_raster(str(inFile),
             band=inHelper.getRgbIndexList(),
             vmin=inHelper._minValue,
             vmax=inHelper._maxValue,
             nodata=inHelper._noDataValue,
             opacity=0.5,
             layer_name=inFile.name
            )

m.add_raster(str(labelsFile),
             vmin=lHelper._minValue,
             vmax=lHelper._maxValue,
             nodata=lHelper._noDataValue,
             opacity=0.5,
             layer_name=labelsFile.name,
             palette='viridis',
            )

m

Map(center=[66.346511472519, -140.3222662042015], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

## Update the labels.
Select multiple values by clicking the mouse or using the arrow keys while pressing shirt, control or command.

In [11]:
opts = list(numpy.unique(labels))

sl = ipywidgets.SelectMultiple( \
        options=opts, 
        layout=(ipywidgets.Layout(height='200px', width='150px')))

bt = ipywidgets.ToggleButtons(options=['Select:', 'Next', 'Done', 'Start Over'], 
                              value='Select:')

output = ipywidgets.Output()
display(sl, bt, output)
table = {}
bt.observe(handleClick, names='value')

SelectMultiple(layout=Layout(height='200px', width='150px'), options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

ToggleButtons(options=('Select:', 'Next', 'Done', 'Start Over'), value='Select:')

Output()

## Edit the groups.
Edit cluster IDs in each group.  When finished, proceed to the next cell.

In [15]:
strTab = {}

for item in table:
    strTab[item] = ', '.join(str(i) for i in table[item])

df = pandas.DataFrame(strTab.items(), columns=['Class', 'Cluster ID'])
sheet = ipysheet.from_dataframe(df)
sheet.column_width = [1, 5]
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=2, row_start=0, squeeze_row=Fa…

In [17]:
editedDf = ipysheet.to_dataframe(sheet)
strClusters = editedDf.to_dict()['Cluster ID']

finalClusters = {}

for key in strClusters:
    
    strCluster = strClusters[key]
    finalClusters[int(key)] = [int(i) for i in strCluster.split(',') if i]

newClusters = relabel(labels, finalClusters)

## Review the updated map.

In [18]:
clusterMapFile = outDirectory / (inFile.stem + '-cluster-map' + inFile.suffix)

cmDataset = Clusterer.labelsToGeotiff(inHelper._dataset, 
                                      clusterMapFile, newClusters)

cmHelper = ImageHelper()
cmHelper.initFromDataset(cmDataset, noDataValue)

m.add_raster(str(clusterMapFile),
             band=cmHelper.getRgbIndexList(),
             vmin=cmHelper._minValue,
             vmax=cmHelper._maxValue,
             nodata=cmHelper._noDataValue,
             opacity=0.5,
             layer_name=clusterMapFile.name
            )

display(m)


Markdown(f'<h3>The cluster map is at<br><br>{clusterMapFile}</h3>')

Map(bottom=129936.0, center=[66.9030773436267, -138.87666489579095], controls=(ZoomControl(options=['position'…

<h3>The cluster map is at<br><br>/explore/nobackup/people/rlgill/innovation-lab-repositories/ImageCluster/my4326-cluster-map.tif</h3>